# HMM(隐马尔可夫模型)
## 隐马尔可夫模型的基本概念
### 背景知识
**动态模型（hmm， Karman Filter, Particle Filter）**包括时间序列和混合模型（系统状态(system state)和观测变量）.

如果系统状态为离散，则模型为HMM；若为线性连续，则模型为Karman Filter;若为非线性连续，则为Particle Filter.
### 隐马尔可夫模型的定义
**状态序列（state sequence）**：隐藏的马尔可夫链随机生成的状态的序列。

**观测序列（observation sequence）**：每个状态生成一个观测，由此产生的观测的随机序列。

设Q为所有可能的状态集合，V是所有可能的观测集合。
$$
Q=\{ q_1,q_2,\cdots,q_N\}, \quad V=\{ v_1,v_2,\cdots,v_N\}
$$

**状态转移矩阵**:$A=[a_{ij}]_{N\times N}$

其中，
$$
a_{ij} = P(i_{t+1}=q_j|i_t = q_i)\quad i=1,2,\cdots,N;j=1,2,\cdots,N
$$
即时刻t处于状态$q_i$的条件下在时刻t+1转移到状态$q_j$的概率。

**观测概率矩阵(发射矩阵)**：$B=[b_j(k)]_{N\times N}$

其中，
$$
b_j(k) = P(o_t=v_k|i_t=q_j)\quad j=1,2,\cdots,N;k=1,2,\cdots,N
$$
即刻t处于状态$q_j$的条件下生成观测$v_k$的概率。

**初始概率向量**：$\pi=(\pi_i)$

其中，
$$
\pi_i = P(i_1=q_i)\quad i=1,2,\cdots,N
$$
即时刻t=1时处于状态$q_i$的概率。

综上，**隐马尔可夫模型**$\lambda$可以表达为
$$
\lambda = (\pi, A, B)
$$
**其次马尔科夫假设（无后效性）**假设隐藏的马尔科夫链在任意时刻链t的状态只依赖其前一时刻的状态，与其他时刻的状态及观测无关，也与时刻t无关。
$$
P(i_t|i_{t-1}, o_{t-1},i_{t-2},o_{t-2}, \cdots,i_1, o_1)=P(i_t|i_{t-1})
$$
**独立观测假设**假设任意时刻的观测只依赖于该时刻的隐马尔科夫链的状态，与其他状态和观测无关。
$$
P(o_t|i_{T}, o_{T},i_{T-1},o_{T-1}, \cdots,i_1, o_1)=P(o_t|i_t)
$$
### 观测序列的生成过程
输入：隐马尔可夫模型$\lambda =(\pi,A,B)$,观测序列长度T；

输出：观测序列$o={o_1,o_2,\cdots,o_T}$

(1)按照初始状态$\pi$产生状态$i_1$

(2)令t=1

(3)按照状态$i_t$的观测概率分布$b_{i_t}(k)$生成$o_t$

(4)按照状态$i_t$的状态转移概率分布$a_{i_t,i_{t+1}}$生成$i_{t+1}$

(5)令t=t+1，若t<T则转(3),否则终止。

### HMM的三个问题
（1）**概率计算问题**。给定模型$\lambda=(\pi,A,B)$和观测序列$O=(o_1,o_2,\cdots,o_T)$,计算模型$\lambda$在观测序列O出现的概率$P(O|\lambda)$.

（2）**学习问题**。已知观测序列$O=(o_1,o_2,\cdots,o_T)$,估计模型$\lambda=(\pi,A,B)$参数是的在该模型下观测序列概率$P(O|\lambda)$最大，即用极大似然估计的方法估计参数。

（3）**预测问题**（解码问题）。已知模型$\lambda=(\pi,A,B)$和观测序列$O=(o_1,o_2,\cdots,o_T)$，求对给定观测序列条件概率P(I|O)最大的状态序列$I=(i_1,i_2,\cdots,i_T)$.即给定观测序列，求最有可能的对应的状态序列。

## 概率计算算法
### 直接计算法
$P(O|\lambda) = \sum_IP(O,I|\lambda)P(I|\lambda)\\
=\sum_{i_1,i_2,\cdots,i_T}\pi_{i_1}b_{i_1}a_{i_1,i_2}\pi_{i_1}b_{i_1}(o_1)a_{i_1,i_2}b_{i_2}(o_2)\cdots a_{i_{T-1},i_T}b_{i_T}(o_T)\\
=\sum_{i_1}\sum_{i_2}\cdots\sum_{i_T}\pi(a_{i1})\prod_{t=2}a_{i_{t-1}}a_{i_t}\prod_{t=1}b_{i_t}(o_t)$

算法复杂度：$O(TN^T)$

### 前向算法
**前向概率**给定HMM$\lambda$，定义到时刻t部分观测序列为$o_1,o_2,\cdots,o_t$且状态为$q_i$的概率为前向概率。
$$
\alpha_t(i)=P(o_1,o_2,\cdots,o_t,i_t=q_i|\lambda)
$$
由此可得
$$
\alpha_{t+1}(j)=P(o_1,o_2,\cdots,o_{t+1},i_{t+1}=q_j|\lambda)\\
=\sum_{i=1}^NP(o_1,o_2,\cdots,o_{t+1},i_{t+1}=q_j, i_t=q_i|\lambda)\\
=\sum_{i=1}^NP(o_{t+1}|o_1,o_2,\cdots,o_t,i_{t+1}=q_j, i_t=q_i,\lambda)*P(o_1,o_2,\cdots,o_t,i_{t+1}=q_j, i_t=q_i|\lambda)\\
=\sum_{i=1}^NP(o_{t+1}|i_{t+1}=q_j, \lambda)*P(o_1,o_2,\cdots,o_t,i_{t+1}=q_j, i_t=q_i|\lambda)\\
=\sum_{i=1}^NP(o_{t+1}|i_{t+1}=q_j, \lambda)*P(i_{t+1}=q_j|o_1,o_2,\cdots,o_t,i_t=q_i,\lambda)*P(o_1,o_2,\cdots,o_t,i_t=q_i|\lambda)\\
=\sum_{i=1}^NP(o_{t+1}|i_{t+1}=q_j, \lambda)*P(i_{t+1}=q_j|i_t=q_i,\lambda)*P(o_1,o_2,\cdots,o_t,i_t=q_i|\lambda)\\
=\sum_{i=1}^N\alpha_ta_{ij}b_j(o_{t+1})
$$
**算法**：

输入：HMM$\lambda$,观测序列O。

输出：观测序列概率$P(O|\lambda)$。

(1)初值$\alpha_1(i)=\pi_ib_i(o_1)$

(2)递推， 对t=1,2,...,N-1,
$$\alpha_{t+1}(j)=\sum_{i=1}^N\alpha_ta_{ij}b_j(o_{t+1})$$

(3)终止，$P(O|\lambda)=\sum_{i=1}^N\alpha_T(i)$

### 后向算法
记$\beta_t(i)=P(o_{t+1,\cdots,o_T|i_t=q_i,\lambda})$ 

$$
P(O|\lambda)=P(o_1,o_2,\cdots,o_T|\lambda)\\
=\sum{i=1}^NP(o_1,o_2,\cdots,o_T,i_1=q_i)\\
=\sum{i=1}^NP(o_1,o_2,\cdots,o_T|i_1=q_i)P(i_1=q_i)\\
\sum{i=1}^NP(o_1|i_1=q_i)P(i_1=q_i)\\
=\sum_{i=1}^N\pi_ib_i(o_1)\beta_1(i)
$$
由递推得(由于HMM给定，推导时进行忽略)
$$
\beta_t(i)=P(o_{t+1},\cdots,o_T|i_t=q_i)\\
=\sum_{i=1}^NP(o_{t+1},\cdots,o_T,i_{t+1}=q_j|i_t=q_i)\\
=\sum_{i=1}^NP(o_{t+1},\cdots,o_T|i_t=q_i,i_{t+1}=q_j)P(i_{t+1}=q_j|i_t=q_i)\\
=\sum_{i=1}^NP(o_{t+1},\cdots,o_T|i_{t+1}=q_j)a_{ij}\\
=\sum_{i=1}^NP(o_{t+1}|o_{t+2},\cdots,o_T,i_{t+1}=q_j)P(o_{t+2},\cdots,o_T|i_{t+1}=q_j)a_{ij}\\
=\sum_{j=1}^Na_{ij}b_j(o_{t+1})\beta_{t+1}(j)
$$

**算法**

输入：HMM$\lambda$,观测序列O。

输出：观测序列概率$P(O|\lambda)$。

(1)
$$
\beta_T(i)=1\quad i=1,2,\cdots,N
$$

(2)对$t=T-1,T-2,\cdots,1$
$$
\beta_t(i)=\sum_{j=1}^Na_{ij}b_j(o_{t+1})\beta_{(t+1)}(j)
$$

(3)
$$
P(O|\lambda)=\sum_{i=1}^N\pi_ib_i(o_1)\beta_1(i)
$$


## 学习算法
### Baum-Welch算法（EM算法）
由EM算法得
$$
\lambda^{(t+1)}=arg\max_{\lambda}\sum_IlogP(O,I|\lambda)P(I|O,\hat{\lambda})
$$
**1.确定完全数据的对数似然函数**
给定训练数据包含S个长度为T的观测序列$\{O_1,O_2,\cdots,O_S\}$,训练目标为HMM$\lambda=(\pi,A,B)$

所有观测数据写为$O=\{o_1,o_2,\cdots,o_T\}$,所有隐数据写为$I=\{i_1,i_2,\cdots,i_T\}$.完全数据$(O,I)=(o-1,\cdots,o_T,i_1,\cdots,i_T)$，完全数据的对数似然函数为$log(O,I|\lambda)$

**2.EM算法E步，求Q函数$Q(\lambda,\hat{\lambda})$**
$$
Q(\lambda,\hat{\lambda})=\sum_IlogP(O,I|\lambda)P(I|O,\hat{\lambda})\\
=\sum_Ilog\pi_{i_1}P(O,I|\hat{\lambda})+\sum_I\left(\sum_{(t=1)}^{T-1}loga_{i,i+1}\right)P(O,I|\hat{\lambda})+\sum_I\left(\sum_{(t=1)}^{T}logb_{i_t}(o_t)\right)P(O,I|\hat{\lambda})
$$

**EM算法M步，极大化Q函数求模型参数**
要极大化的三项分别单独出现在三项中，因此只需要对各项分别极大化
**(1)**
Q函数第一项可以改写为
$$
\sum_Ilog\pi_{i_1}P(O,I|\hat{\lambda})=\sum_{i=1}^Nlog\pi_iP(O,i_1=i|\hat{\lambda})
$$
注意到$\sum_{i=1}^N\pi_i=1$,利用拉格朗日乘子法，得到拉格朗日函数
$$
\sum_{i=1}^Nlog\pi_iP(O,i_1=i|\hat{\lambda})+\gamma\left(\sum_{i=1}^N\pi_i-1\right)
$$
对$\pi_i$求偏导并令结果为0得
$$
P(O,i_1=i|\hat{\lambda})+\gamma\pi_i=0
$$
对i求和得到
$$
\gamma=-P(O|\hat{\lambda})
$$
即
$$
\pi_i=\frac{P(O,i_1=i|\hat{\lambda})}{P(O|\hat{\lambda})}
$$

**(2)**
Q函数第二项可以改写为
$$
\sum_I\left(\sum_{(t=1)}^{T-1}loga_{i,i+1}\right)P(O,I|\hat{\lambda})=\sum_{i=1}^N\sum_{j=1}^N\sum_{t=1}^{T-1}loga_{ij}P(O,i_t=i,i_{t+1}=j|\hat{\lambda})
$$
注意到约束条件$\sum_{j=1}^Na_{ij}=1$,同(1)得
$$
a_{ij}=\frac{\sum_{t=1}^{T-1}P(O,i_t=i,i_{t+1}=j|\hat{\lambda})}{\sum_{t=1}^{T-1}P(O,i_t=i|\hat{\lambda}}
$$

**(3)**
Q函数第三项可改写为
$$
\sum_I\left(\sum_{t=1}^{T}logb_{i_t}(o_t)\right)P(O,I|\hat{\lambda})=\
\sum_{j=1}^N\sum_{t=1}^{T}logb_{j}(o_t)P(O,i_t=j|\hat{\lambda})
$$
由约束$\sum_{k=1}^Mb_j(k)=1$得
$$
b_j(k)=\frac{\sum_{t=1}^TP(O,i_t=j|\hat{\lambda})I(o_t=v_k)}{\sum_{t=1}^TP(O,i_t=j|\hat{\lambda})}
$$

综上，若将$\gamma_t(i):=\frac{P(O,i_t=i|\hat{\lambda})}{P(O|\hat{\lambda})},\quad \zeta_t(i,j):=\
P(i_t=i,i_{t+1}=j|O,\hat{\lambda})$,则上述公式可以表示为
$$
a_{ij}=\frac{\sum_{t=1}^{T-1}\zeta_t(i,j)}{\sum_{t=1}^{T-1}\gamma_t(i)}\\
b_j(k)=\frac{\sum_{t=1,o_t=v_k}^{T}\gamma_t(j)}{\sum_{t=1}^{T-1}\gamma_t(j)}\\
\pi_i=\gamma_1(i)
$$

**算法**
输入：观测数据$O=(o_1,o_2,\cdots,o_T)$;
输出：HMM
(1)初始化

对n=0，选取$a_{ij}^{(0)},b_j(k)^{(0)},\pi_i^{(0)}$得到模型$\lambda^{(0)}=(\pi^{(0)},A^{(0)},B^{(0)}$

(2)递推，对$n=1,2,\cdots,$
$$
a_{ij}^{(n+1)}=\frac{\sum_{t=1}^{T-1}\zeta_t(i,j)}{\sum_{t=1}^{T-1}\gamma_t(i)}\\
b_j(k)^{(n+1)}=\frac{\sum_{t=1,o_t=v_k}^{T}\gamma_t(j)}{\sum_{t=1}^{T-1}\gamma_t(j)}\\
\pi_i^{(n+1)}=\gamma_1(i)
$$

(3)终止，得到HMM







## 预测算法
### 近似算法
给定HMM$\lambda$和观测序列O，在t时刻处于状态$q_i$的概率$\gamma_t(i)$为
$$
\gamma_t(i)=\frac{\alpha_t(i)\beta_t(i)}{P(O|\lambda)}=\frac{\alpha_t(i)\beta_t(i)}{\sum_{j=1}^N\alpha_t(j)\beta_t(j)}
$$
在每个时刻t最可能的状态$i^*_t$是
$$
i^*_t=arg\max_{i\le i\le N}[\gamma_t^i],t=1,2,\cdot,T
$$
从而得到序列$I*=(i^*_1,i^*_2,\cdots,i^*_T)$
### 维特比算法
输入：模型$\lambda$和观测$O=(o_1,o_2,\cdots,o_T)$;

输出：最优路径$I*=(i^*_1,i^*_2,\cdots,i^*_T)$

(1)初始化
$$
\delta_1(i)=\pi_ib_i(o_1),i=1,2,\cdots,N\\
\Phi_1(i)=0,i=1,2,\cdots,N
$$

(2)递推，对$t=2,3,\cdots,T$
$$
\delta_t(i)=\max_{1\le j\le N}[\delta_{t-1}(j)a_{ji}]b_i(o_t),i=1,2,\cdots,N\\
\Phi_t(i)=arg\max_{1\le j\le N}[\delta_{t-1}(j)a_{ji}],i=1,2,\cdots,N
$$

(3)终止
$$
P^*=\max_{1\le j\le N}\delta_T(i)\\
i_T^*=arg\max_{1\le j\le N}[\delta_T(i)]
$$